In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [2]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta
import time
import abc

from data import TDDataFetcher

import mplfinance as mpf
# import ruptures as rpt

importing Jupyter notebook from data.ipynb


<string>:114: SyntaxWarning: "is" with a literal. Did you mean "=="?


mu = 0.002005
sigma = 0.023731
ABMD
                High     Low    Open   Close   Volume        5EMA        8EMA  \
Date                                                                            
2020-01-02  173.8600  167.14  172.51  168.81   734344  137.314496  131.227039   
2020-01-03  169.3174  165.24  166.83  166.82   746102  147.149664  139.136586   
2020-01-06  179.4200  166.02  166.53  179.04  1506438  157.779776  148.004011   
2020-01-07  182.4400  177.19  178.83  180.35   991684  165.303184  155.192009   
2020-01-08  180.7300  174.29  178.38  178.69  1122415  169.765456  160.413784   
...              ...     ...     ...     ...      ...         ...         ...   
2021-12-03  316.6200  295.71  314.35  300.63   275988  310.960784  316.754593   
2021-12-06  310.2500  297.22  302.07  308.15   306220  310.023856  314.842461   
2021-12-07  326.7900  314.23  314.23  320.78   410958  313.609238  316.161914   
2021-12-08  326.4100  316.87  321.50  324.41   194075  317.209492  317.99

In [3]:
class Asset(metaclass=abc.ABCMeta):
    """
        Abstract base cass for representing a purchased 
    """
    def __init__(self):
        pass
    
class Stock(Asset):
    def __init__(self, ticker : str, data : pd.DataFrame, purchased : datetime):
        super(Stock, self).__init__()
        self.ticker = ticker
        self.data = data
        self._data_start_date = self.data.index[0]
        self._data_end_date = self.data.index[-1]
        self.purchased = purchased

class Portfolio(): 
    def __init__(self, assets : [Asset]):
        self.assets = assets

In [4]:
class AssetModel(metaclass=abc.ABCMeta):
    """
        Abstract base cass for representing an asset (with theoretical value)
    """
    def __init__(self):
        pass
    
class ChartableSecurity(metaclass=abc.ABCMeta):
    def __init__(self, data):
        self.data = data
        self.date_str = datetime.strptime(str(data.index[0]), '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%y') + ' - ' + datetime.strptime(str(data.index[-1]), '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%y')
        self.__mav_colors = mavcolors=['Blue','Orange','Magenta','Brown','Grey']
        self.__style = mpf.make_mpf_style(base_mpf_style='yahoo', y_on_right=False, edgecolor='Black', facecolor='White', mavcolors=self.__mav_colors)
        
    def _plot(self, data, type_name='candle', desc='', apds=[], mav=[], hlines=[], add_plots=[]):
#         add_plots = [mpf.make_addplot(self.__add_mav(avg),linestyle='solid') for avg in mav]
#         apds.append(add_plots)
        fig, axs = mpf.plot(data, addplot=add_plots, figscale=1, figratio=(2*8,2*5.75), type=type_name, volume=True, hlines=hlines, style=self.__style, title='\n\n%s %s\n%s' % (self.__name__(), desc, self.date_str), returnfig=True)
        
        for avg in mav:
            axs[0].plot(np.arange(len(data.index)), self.__add_mav(avg))
            
        return fig, axs
    
    def plot_custom_mav(self, mavs, type_name='line'):
        add_plots = [mpf.make_addplot(mav,linestyle='solid') for mav in mavs]

        return self._plot(self.data, desc="Close Price (EMA)", type_name=type_name, add_plots=add_plots)

    
    def _append_pivot_points(self, lines=True):
        if lines:
            shift = self.data.shift(1)
            self.data['pivot'] = (shift["High"] + shift["Low"] + shift["Close"])/3.0
            ppt2 = self.data['pivot'] * 2
            self.data['support1'] = ppt2 - self.data['High']
            self.data['support2'] = self.data['pivot'] - self.data['High'] + self.data['Low']
            self.data['resistance1'] = ppt2 - shift['Low']
            self.data['resistance2'] = self.data['pivot'] + self.data['High'] - self.data['Low']

            ppts = ['pivot', 'support1', 'support2', 'resistance1', 'resistance2']
            add_plots = [mpf.make_addplot(self.data[ppt],linestyle='solid') for ppt in ppts]
            return add_plots
        else:
            pass
            
    
    def plot_close_price(self, mav=[], support_resistance=False): 
        if support_resistance:
            add_plots = self._append_pivot_points(lines=False)            
            fig, axs = self._plot(self.data, desc='Close Price', type_name='line', mav=mav, add_plots=add_plots)
        else:
            hl = dict(hlines=[250,300], colors=['g','r'],linestyle='-.')
            fig, axs = self._plot(self.data, desc='Close Price', type_name='line', mav=mav,)
        return fig, axs
        
    def plot_candlesticks(self, mav=[]):            
        fig, axs = self._plot(self.data, type_name='candle', mav=mav)
        return fig, axs
        
    def plot_ohlc(self, mav=[]):
        fig, axs = self._plot(self.data, type_name='ohlc', mav=mav)
        return fig, axs
        
    def __add_mav(self, time_frame : int):
        if not 'MA%d' % time_frame in self.data:
            self.data.insert(len(self.data.columns),'MA%d' % time_frame, 0)
            self.data['MA%d' % time_frame] = self.data['Close'].rolling(time_frame).mean()
            
        return self.data['MA%d' % time_frame].values
        
#     def plot_renko(self): # TODO
#         fig, axs = self._plot(self.data, desc='(Renko)', type_name='renko')

    def plot_prediction(self, prediction, summary=False, type_name='line', desc='Prediction', adps=[], mavs=[]):
        index = self.data.index.append(prediction.index)
        prediction = pd.DataFrame(prediction, index=index)
        final_observed = aapl.data.iloc[-1]
        prediction.loc[final_observed.name] = final_observed['Close']
        
        if summary:            
            add_plots = [mpf.make_addplot(prediction.mean(axis=1),linestyle='solid'),
                mpf.make_addplot(prediction.max(axis=1),linestyle='solid'),
                mpf.make_addplot(prediction.min(axis=1),linestyle='solid')
            ]
        else:
            add_plots = [mpf.make_addplot(prediction[path],color='red', linestyle='solid') for path in prediction]

        data_trans = pd.DataFrame(self.data, index=index)
        fig, axs = mpf.plot(data_trans, addplot=add_plots, figscale=1, figratio=(2*8,2*5.75), type=type_name, volume=False, style=self.__style, title='\n\n%s %s\n%s' % (self.__name__(), desc, self.date_str), returnfig=True)
                
        
    @abc.abstractproperty
    def __name__(self):
        pass



In [5]:
class StockModel(AssetModel, ChartableSecurity):
    def __init__(self, ticker : str, data : pd.DataFrame):
        AssetModel.__init__(self)
        ChartableSecurity.__init__(self, data)
        self.ticker = ticker
        self._data_start_date = self.data.index[0]
        self._data_end_date = self.data.index[-1]
        self.data.insert(len(self.data.columns),'Return', 0)
        self.data['Return'] = self.data['Close'].pct_change()

    def __name__(self):
        return self.ticker
        
    def __str__(self):
        return self.ticker + ":\n" + str(self.data)
    
    def predict_GBM(self, T, dt=1, M=1):
        """
        Parameters:
            T --> time horizon in days
            dt --> time increment (defaults to 1 day)
            M --> # of paths to simulate
        """
        S0 = self.data.iloc[0]['Close']
        St = self.data.iloc[-1]['Close']
        mu = np.mean(self.data["Return"])
        sigma = np.std(self.data['Return'])
        
        N = T / dt
        indicies = pd.date_range(start=self.data.iloc[-1].name + timedelta(days=1), end=self.data.iloc[-1].name + timedelta(days=N), freq='D').map(lambda x : x if x.isoweekday() in range (1,6) else np.nan).dropna()
        N = len(indicies)
        
        drift = (mu - .5 * sigma * sigma)
        logS0 = np.log(St)
        
        df = pd.DataFrame(np.nan, columns=[], index=indicies)

        for i in range(M):
            
            logS = logS0
            diffusion = sigma * np.sqrt(dt)
            pred = np.empty(int(N))

            for j in range(int(N)):
                logS = logS + drift * dt + diffusion * np.random.normal(0,1)
                pred[j] = np.exp(logS)
                
            df.insert(len(df.columns),'Path%d' % (i+1), pred)
        
        return df

In [6]:
class Basket(metaclass=abc.ABCMeta): 
    def __init__(self, assets : [AssetModel]):
        self.assets = assets
        
    def __str__(self):
        string = ""
        for asset in self.assets:
            string += str(asset) + "\n\n\n"
        return string
    
class AssetGenerator(metaclass=abc.ABCMeta):
    """
        Abstract base cass for representing an asset
    """
    def __init__(self):
        pass

# TODO : Make StockGenerator and generalize to StockLoader
class StockFetcher(AssetGenerator):
    def __init__(self, data_gen): # TODO : generalized to DataLoader then DataGenerator
        super(StockFetcher, self).__init__()
        self.tickers = data_gen.get_tickers()
        self.data_gen = data_gen 
             
    # TODO : note that data_gen holds on to data after returns
    
    def get_historic_daily_quotes(self, start_date : datetime, end_date=date.today()):
        self.data_gen.get_daily_price_history(start_date, end_date)        
        return Basket([StockModel(ticker, self.data_gen.get_data_for(ticker)) for ticker in self.tickers])
            
    def get_historic_monthly_quotes(self, start_date : datetime, end_date=date.today()):
        self.data_gen.get_daily_price_history(start_date, end_date)        
        return Basket([StockModel(ticker, self.data_gen.get_data_for(ticker)) for ticker in self.tickers])
    
    def get_historic_yearly_quotes(self, start_date : datetime, end_date=date.today()):
        self.data_gen.get_yearly_price_history(start_date, end_date)        
        return Basket([StockModel(ticker, self.data_gen.get_data_for(ticker)) for ticker in self.tickers])
    
class TDStockFetcher(StockFetcher):
    def __init__(self, tickers):
        super(TDStockFetcher, self).__init__(TDDataFetcher(tickers))

In [7]:
portfolio = ['AAPL', 'MSFT', 'GOOG', "AMZN"]
stock_fetcher = TDStockFetcher(portfolio).get_historic_daily_quotes(datetime(2019, 6, 1))
aapl = stock_fetcher.assets[0]
aapl.data
aapl.plot_close_price(support_resistance=True)

<ipython-input-5-1746354b3b3e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Return'] = self.data['Close'].pct_change()


,High,Low,Open,Close,Volume,Return
Date,,,,,,
2019-06-03,44.480000,42.567500,43.900000,43.32500,161584276,NaN
2019-06-04,44.957500,43.630000,43.860000,44.91000,123871844,0.036584
2019-06-05,46.247500,45.285000,46.070000,45.63500,119093708,0.016143
2019-06-06,46.367500,45.537225,45.770000,46.30500,90105244,0.014682
2019-06-07,47.980000,46.442500,46.627500,47.53750,122737572,0.026617
2019-06-10,48.842500,47.905000,47.952500,48.14500,104883404,0.012779
2019-06-11,49.000000,48.400000,48.715000,48.70250,107731528,0.011580
2019-06-12,48.992500,48.346250,48.487500,48.54750,73012756,-0.003183
2019-06-13,49.197500,48.400000,48.675000,48.53750,86698500,-0.000206


TypeError: kwarg "addplot" validator returned False for value: "None"
    'Validator'   : lambda value: isinstance(value,dict) or (isinstance(value,list) and all([isinstance(d,dict) for d in value])) },

In [ ]:
aapl = StockFetcher(TDDataFetcher(['AAPL'])).get_historic_daily_quotes(datetime(2020, 1, 1)).assets[0]
# pred = aapl.predict_GBM(100, M=1000)
# aapl.plot_prediction(pred, summary=True)
fig, axs = aapl.plot_close_price(support_resistance=False)
# aapl.data


In [ ]:
portfolio = ['AAPL', 'MSFT', 'GOOG', "AMZN"]
stock_fetcher = TDStockFetcher(portfolio).get_historic_daily_quotes(datetime(2019, 6, 1))
aapl = stock_fetcher.assets[0]
# aapl.insert(-1, "15MA" ,0.0)
aapl.data["5EMA"] = aapl.data['Close'].ewm(span=5,adjust=False).mean()
aapl.data["8EMA"] = aapl.data['Close'].ewm(span=8,adjust=False).mean()
aapl.data["13EMA"] = aapl.data['Close'].ewm(span=13,adjust=False).mean()
aapl.data["20EMA"] = aapl.data['Close'].ewm(span=20,adjust=False).mean()
aapl.data["50EMA"] = aapl.data['Close'].ewm(span=50,adjust=False).mean()

# aapl.data["50EMA"] = np.nan
aapl.data

In [ ]:
aapl.plot_custom_mav(mavs=[aapl.data["5EMA"],aapl.data["8EMA"],aapl.data["13EMA"] ])